# [Understanding Factual Errors in Summarization: Errors, Summarizers, Datasets, Error Detectors](https://arxiv.org/abs/2205.12854)



```
@misc{tang2023understanding,
      title={Understanding Factual Errors in Summarization: Errors, Summarizers, Datasets, Error Detectors}, 
      author={Liyan Tang and Tanya Goyal and Alexander R. Fabbri and Philippe Laban and Jiacheng Xu and Semih Yavuz and Wojciech Kryściński and Justin F. Rousseau and Greg Durrett},
      year={2023},
      eprint={2205.12854},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}

```

In [1]:
import pandas as pd
import numpy as np
import sklearn
from utils import choose_best_threshold
import warnings
warnings.filterwarnings('ignore')

from utils import SOTA, XFORMER, OLD, MAPPING

In [2]:
dataset_path="data/aggre_fact_final.csv"
df = pd.read_csv(dataset_path, index_col = 0)

# split data
df_val = df[df.cut == 'val']
df_val_sota = df_val[df_val.model_name.isin(SOTA)]
df_test = df[df.cut == 'test']
df_test_sota = df_test[df_test.model_name.isin(SOTA)]

dataset_list = ['XSumFaith', 'Polytope', 'FactCC', 'SummEval', 'FRANK', 'Wang20', 'CLIFF', 'Goyal21', 'Cao22']
systems = ['DAE', 'QuestEval', 'SummaC-ZS', 'SummaC-Conv', 'QAFactEval', 'FaCED', 'FaCED_wo_GE', 'AlignScore']
origins = ['cnndm', 'xsum']

In [3]:
main_df = pd.DataFrame(
    columns=['system', 'origin', 'count', 'dataset', 'category', 'bl_acc']
)

results = []

for system in systems:
    df[f'{system}_label'] = None

for system in systems:
    for origin in origins:
        for dataset in dataset_list:
            for i, model_novelty in enumerate([SOTA, XFORMER, OLD]):
                df_val_temp = df_val[(df_val.dataset == dataset) & (df_val.origin == origin) & (df_val.model_name.isin(model_novelty))]
                df_test_temp = df_test[(df_test.dataset == dataset) & (df_test.origin == origin) & (df_test.model_name.isin(model_novelty))]
                if len(df_val_temp) > 0 and len(df_test_temp) > 0:
                    best_thresh, best_f1 = choose_best_threshold(df_val_temp.label.values, df_val_temp[f'{system}_score'].values)
                    scores_test = df_test_temp[f'{system}_score'].values
                    preds_test = [1 if score > best_thresh else 0 for score in scores_test]
                    df.loc[df_test_temp.index, f'{system}_label'] = preds_test
                    
                    balanced_acc = sklearn.metrics.balanced_accuracy_score(df_test_temp.label.values, preds_test)

                    main_df.loc[len(main_df.index)] = [
                        system, origin, len(preds_test), dataset, MAPPING[i], balanced_acc
                    ]

                    results.append({"system": system, "dataset_name": dataset, 'origin': origin, 
                    'count': len(scores_test), 'cat': MAPPING[i], "labels": df_test_temp.label.values, 
                    "preds": preds_test, "scores": scores_test})

df = df.reindex(
    columns=['dataset', 'origin', 'id', 'doc', 'summary', 'model_name', 'label',
       'cut', 'DAE_score', 'DAE_label', 'QuestEval_score', 'QuestEval_label',
       'SummaC-ZS_score', 'SummaC-ZS_label', 'SummaC-Conv_score', 'SummaC-Conv_label', 
       'QAFactEval_score' , 'QAFactEval_label'],
)

#  Dataset-wise comparsion between factuality systems

In [4]:
# Table 8
main_df_pivot_bacc = main_df.pivot(index=['origin', 'dataset', 'category', 'count'], columns='system', values='bl_acc')
main_df_pivot_bacc = main_df_pivot_bacc.reindex(columns=systems)
main_df_pivot_bacc.round(3)

system                             DAE  QuestEval  SummaC-ZS  SummaC-Conv  \
origin dataset   category count                                             
cnndm  CLIFF     SOTA     150    0.730      0.740      0.646        0.649   
       FRANK     OLD      523    0.704      0.670      0.692        0.727   
                 SOTA     175    0.699      0.626      0.570        0.601   
                 XFORMER  175    0.574      0.556      0.631        0.634   
       FactCC    OLD      503    0.704      0.655      0.835        0.891   
       Goyal21   OLD      25     0.188      0.146      0.375        0.354   
       Polytope  OLD      450    0.779      0.687      0.802        0.791   
                 SOTA     34     0.294      0.176      0.971        0.735   
                 XFORMER  150    0.774      0.733      0.970        0.811   
       SummEval  OLD      548    0.661      0.649      0.773        0.801   
                 SOTA     200    0.452      0.649      0.622        0.827   
                 XFORMER  50     0.760      0.680      0.620        0.580   
       Wang20    OLD      117    0.586      0.552      0.655        0.672   
xsum   CLIFF     SOTA     150    0.754      0.619      0.596        0.673   
       Cao22     SOTA     239    0.723      0.601      0.490        0.668   
       Goyal21   SOTA     50     0.644      0.814      0.466        0.552   
       Wang20    SOTA     119    0.756      0.560      0.698        0.721   
       XSumFaith OLD      430    0.834      0.597      0.533        0.675   
                 XFORMER  423    0.855      0.601      0.514        0.646   

system                           QAFactEval  FaCED  FaCED_wo_GE  AlignScore  
origin dataset   category count                                              
cnndm  CLIFF     SOTA     150         0.716  0.735        0.785       0.685  
       FRANK     OLD      523         0.773  0.714        0.730       0.759  
                 SOTA     175         0.547  0.692        0.680       0.592  
                 XFORMER  175         0.646  0.649        0.651       0.637  
       FactCC    OLD      503         0.843  0.786        0.802       0.859  
       Goyal21   OLD      25          0.271  0.479        0.479       0.500  
       Polytope  OLD      450         0.824  0.815        0.817       0.887  
                 SOTA     34          0.324  0.794        0.735       0.324  
                 XFORMER  150         0.726  0.666        0.672       0.838  
       SummEval  OLD      548         0.814  0.751        0.785       0.732  
                 SOTA     200         0.652  0.755        0.593       0.406  
                 XFORMER  50          0.740  0.780        0.740       0.740  
       Wang20    OLD      117         0.754  0.750        0.728       0.810  
xsum   CLIFF     SOTA     150         0.626  0.760        0.776       0.752  
       Cao22     SOTA     239         0.613  0.697        0.706       0.668  
       Goyal21   SOTA     50          0.754  0.622        0.584       0.754  
       Wang20    SOTA     119         0.756  0.646        0.612       0.687  
       XSumFaith OLD      430         0.605  0.685        0.698       0.637  
                 XFORMER  423         0.596  0.724        0.715       0.801

# AggreFact-CNN

In [5]:
# Table 4
scores = []
for cat in MAPPING.values():
    score = []
    for system in systems:
        system_df = main_df[(main_df.system == system) & (main_df.category == cat) & (main_df.origin == 'cnndm')]
        value = sum(system_df['count'] * system_df['bl_acc']) / sum(system_df['count'])
        score.append(round(value, 3))
    scores.append(score)

weighted_df = pd.DataFrame(
    scores,
    columns=systems,
    index=['SOTA', 'XFORMER', 'OLD']
)
weighted_df

,DAE,QuestEval,SummaC-ZS,SummaC-Conv,QAFactEval,FaCED,FaCED_wo_GE,AlignScore
SOTA,0.594,0.637,0.633,0.703,0.616,0.732,0.680,0.534
XFORMER,0.679,0.643,0.765,0.698,0.691,0.673,0.671,0.731
OLD,0.697,0.652,0.763,0.790,0.803,0.760,0.776,0.802


# AggreFact-XSum

In [6]:
# Table 4
scores = []
for cat in MAPPING.values():
    score = []
    for system in systems:
        system_df = main_df[(main_df.system == system) & (main_df.category == cat) & (main_df.origin == 'xsum')]
        value = sum(system_df['count'] * system_df['bl_acc']) / sum(system_df['count'])
        score.append(round(value, 3))
    scores.append(score)

weighted_df = pd.DataFrame(
    scores,
    columns=systems,
    index=['SOTA', 'XFORMER', 'OLD']
)
weighted_df

,DAE,QuestEval,SummaC-ZS,SummaC-Conv,QAFactEval,FaCED,FaCED_wo_GE,AlignScore
SOTA,0.731,0.616,0.561,0.670,0.660,0.697,0.694,0.702
XFORMER,0.855,0.601,0.514,0.646,0.596,0.724,0.715,0.801
OLD,0.834,0.597,0.533,0.675,0.605,0.685,0.698,0.637


# AggreFact-CNN/XSum-SOTA

In [4]:
from utils import resample_balanced_acc

main_sota_df = pd.DataFrame(
    columns=['system', 'origin', 'bl_acc']
)

results = []

for system in systems:
    for origin in origins:
        df_val_temp = df_val_sota[(df_val_sota.origin == origin)]
        df_test_temp = df_test_sota[(df_test_sota.origin == origin)]

        best_thresh, best_f1 = choose_best_threshold(df_val_temp.label.values, df_val_temp[f'{system}_score'].values)
        scores_test = df_test_temp[f'{system}_score'].values
        preds_test = [1 if score > best_thresh else 0 for score in scores_test]

        f1_score = sklearn.metrics.balanced_accuracy_score(df_test_temp.label.values, preds_test)

        main_sota_df.loc[len(main_sota_df.index)] = [
            system, origin, f1_score
        ]

        results.append({"system": system, 'origin': origin,  "labels": df_test_temp.label.values, 
        "preds": preds_test, "scores": scores_test})

In [5]:
# Table 5
# standard deviation may differ due to randomness

# from https://github.com/tingofurro/summac/
P5 = 5 / 2 # Correction due to the fact that we are running 2 tests with the same data
P1 = 1 / 2 # Correction due to the fact that we are running 2 tests with the same data

for origin in origins:
    sampled_batch_preds = {res["system"]: [] for res in results}
    
    for res in results:
        if res['origin'] == origin:
    
            samples = resample_balanced_acc(res["preds"], res["labels"])
            sampled_batch_preds[res["system"]].append(samples)
            low5, high5 = np.percentile(samples, P5), np.percentile(samples, 100-P5)
            low1, high1 = np.percentile(samples, P1), np.percentile(samples, 100-P1)
            bacc = sklearn.metrics.balanced_accuracy_score(res["labels"], res["preds"])

            print(res['origin'].center(6), res["system"].center(20), " - %.3f, %.3f" % (bacc, bacc-low5))
    print()

cnndm          DAE           - 0.654, 0.035
cnndm       QuestEval        - 0.702, 0.030
cnndm       SummaC-ZS        - 0.640, 0.031
cnndm      SummaC-Conv       - 0.610, 0.030
cnndm       QAFactEval       - 0.678, 0.035
cnndm      T-CTRF-CDoc       - 0.691, 0.030
cnndm     T-CTRF-CDoc-3G     - 0.711, 0.032

 xsum          DAE           - 0.702, 0.018
 xsum       QuestEval        - 0.595, 0.019
 xsum       SummaC-ZS        - 0.564, 0.013
 xsum      SummaC-Conv       - 0.650, 0.020
 xsum       QAFactEval       - 0.639, 0.018
 xsum      T-CTRF-CDoc       - 0.679, 0.018
 xsum     T-CTRF-CDoc-3G     - 0.693, 0.018

